# Fundamentals of Information Systems

## Python Programming (for Data Science)

### Master's Degree in Data Science

#### Giorgio Maria Di Nunzio
#### (Courtesy of Gabriele Tolomei FIS 2018-2019)
<a href="mailto:giorgiomaria.dinunzio@unipd.it">giorgiomaria.dinunzio@unipd.it</a><br/>
University of Padua, Italy<br/>
2021/2022<br/>

# Lecture 13: The Classification Problem - Example (Part 2)

## Instructions

-  We consider the dataset file <code>**dataset.csv**</code>, which is contained in the <code>**loan-prediction**</code> directory on the Moodle page.

-  A description of the dataset is available in the <code>**README.txt**</code> file on the same directory.

-  **GOAL:** Use information from past loan applicants contained in <code>**dataset.csv**</code> to predict whether a _new_ applicant should be granted a loan or not.

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Import stats module from scipy, which contains a large number 
# of probability distributions as well as an exhaustive library of statistical functions.
import scipy.stats as stats

%matplotlib inline

## Summary of Part 1

In [2]:
# Path to the local dataset file
DATASET_PATH = "./data/loan-prediction/dataset.csv"

## Loading the Dataset

In [3]:
# Load the dataset with Pandas
data = pd.read_csv(DATASET_PATH, sep=",", index_col="Loan_ID")
print("Shape of the dataset: {}".format(data.shape))
data.head()
# NOTE: the first line of the file is considered as the header

Shape of the dataset: (614, 12)


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


## Handling Missing Values (NA)

In [4]:
# is_numeric_dtype(pandas.Series) returns True iff the dtype associated with the pandas.Series is numeric
from pandas.api.types import is_numeric_dtype

data = data.apply(lambda x: x.fillna(x.median()) 
                      if is_numeric_dtype(x) 
                      else x.fillna(x.mode().iloc[0]))

## Handling Outliers

In [5]:
# Let's winsorize 'ApplicantIncome', 'CoapplicantIncome', and 'LoanAmount'
stats.mstats.winsorize(data.ApplicantIncome, limits=0.05, inplace=True)
stats.mstats.winsorize(data.CoapplicantIncome, limits=0.05, inplace=True)
stats.mstats.winsorize(data.LoanAmount, limits=0.05, inplace=True)

# Apply log-transformation to 'ApplicantIncome' and assign it to a new column
data['Log_ApplicantIncome'] = data.ApplicantIncome.apply(np.log)
# Apply log-transformation to 'LoanAmount' and assign it to a new column
data['Log_LoanAmount'] = data.LoanAmount.apply(np.log)

## Encoding Categorical Features: One-Hot Encoding

In [6]:
# In pandas we can achieve easily one-hot encoding using the 'get_dummies()' function
categorical_features = [col for col in data.columns if not is_numeric_dtype(data[col]) and col != 'Loan_Status']
data_with_dummies = pd.get_dummies(data, columns = categorical_features)
data_with_dummies.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Log_ApplicantIncome,Log_LoanAmount,Gender_Female,Gender_Male,...,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,Y,8.674026,4.852030,False,True,...,False,False,False,True,False,True,False,False,False,True
LP001003,4583,1508.0,128.0,360.0,1.0,N,8.430109,4.852030,False,True,...,True,False,False,True,False,True,False,True,False,False
LP001005,3000,0.0,66.0,360.0,1.0,Y,8.006368,4.189655,False,True,...,False,False,False,True,False,False,True,False,False,True
LP001006,2583,2358.0,120.0,360.0,1.0,Y,7.856707,4.787492,False,True,...,False,False,False,False,True,True,False,False,False,True
LP001008,6000,0.0,141.0,360.0,1.0,Y,8.699515,4.948760,False,True,...,False,False,False,True,False,True,False,False,False,True


In [7]:
# Just as a convention, I prefer to place the column to be predicted
# as the last one.
columns = data_with_dummies.columns.tolist()

# Popping out 'mpg' from the list and insert it back at the end.
columns.insert(len(columns), columns.pop(columns.index('Loan_Status')))

# Let's refactor the DataFrame using this new column index
data_with_dummies = data_with_dummies.loc[:, columns]
data_with_dummies.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Log_ApplicantIncome,Log_LoanAmount,Gender_Female,Gender_Male,Married_No,...,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,8.674026,4.852030,False,True,True,...,False,False,True,False,True,False,False,False,True,Y
LP001003,4583,1508.0,128.0,360.0,1.0,8.430109,4.852030,False,True,False,...,False,False,True,False,True,False,True,False,False,N
LP001005,3000,0.0,66.0,360.0,1.0,8.006368,4.189655,False,True,False,...,False,False,True,False,False,True,False,False,True,Y
LP001006,2583,2358.0,120.0,360.0,1.0,7.856707,4.787492,False,True,False,...,False,False,False,True,True,False,False,False,True,Y
LP001008,6000,0.0,141.0,360.0,1.0,8.699515,4.948760,False,True,True,...,False,False,True,False,True,False,False,False,True,Y


## Encoding Binary Class Label

In [8]:
data = data_with_dummies
data.Loan_Status = data.Loan_Status.map(lambda x: 1 if x=='Y' else -1)
data.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Log_ApplicantIncome,Log_LoanAmount,Gender_Female,Gender_Male,Married_No,...,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,8.674026,4.852030,False,True,True,...,False,False,True,False,True,False,False,False,True,1
LP001003,4583,1508.0,128.0,360.0,1.0,8.430109,4.852030,False,True,False,...,False,False,True,False,True,False,True,False,False,-1
LP001005,3000,0.0,66.0,360.0,1.0,8.006368,4.189655,False,True,False,...,False,False,True,False,False,True,False,False,True,1
LP001006,2583,2358.0,120.0,360.0,1.0,7.856707,4.787492,False,True,False,...,False,False,False,True,True,False,False,False,True,1
LP001008,6000,0.0,141.0,360.0,1.0,8.699515,4.948760,False,True,True,...,False,False,True,False,True,False,False,False,True,1


# 4. Building a Predictive Model

In [12]:
# from sklearn.metrics import SCORERS
import sklearn.metrics
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn import tree
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import explained_variance_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
#from sklearn.externals import joblib

## 4.1 Splitting the Dataset: _Training_ vs. _Test_

In [13]:
"""
Extract the feature matrix from our original DataFrame.
"""
# Feature matrix X is composed of all the columns 
# except 'Loan_Status' (i.e., the target class label)
X = data.iloc[:, :-1]
X.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Log_ApplicantIncome,Log_LoanAmount,Gender_Female,Gender_Male,Married_No,...,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,8.674026,4.852030,False,True,True,...,False,False,False,True,False,True,False,False,False,True
LP001003,4583,1508.0,128.0,360.0,1.0,8.430109,4.852030,False,True,False,...,True,False,False,True,False,True,False,True,False,False
LP001005,3000,0.0,66.0,360.0,1.0,8.006368,4.189655,False,True,False,...,False,False,False,True,False,False,True,False,False,True
LP001006,2583,2358.0,120.0,360.0,1.0,7.856707,4.787492,False,True,False,...,False,False,False,False,True,True,False,False,False,True
LP001008,6000,0.0,141.0,360.0,1.0,8.699515,4.948760,False,True,True,...,False,False,False,True,False,True,False,False,False,True


In [14]:
"""
Similarly, we want to extract the target class column vector y.
"""
y = data.Loan_Status
y.head()

Loan_ID
LP001002    1
LP001003   -1
LP001005    1
LP001006    1
LP001008    1
Name: Loan_Status, dtype: int64

In [15]:
"""
Let's split our dataset with scikit-learn 'train_test_split' function, 
which splits the input dataset into training and test set, respectively.
We want the training set to account for 80% of the original dataset, whilst 
the test set to account for the remaining 20%.
Additionally, we would like to take advantage of stratified sampling,
so as to obtain the same target distribution in both the training and the test sets.
"""

"\nLet's split our dataset with scikit-learn 'train_test_split' function, \nwhich splits the input dataset into training and test set, respectively.\nWe want the training set to account for 80% of the original dataset, whilst \nthe test set to account for the remaining 20%.\nAdditionally, we would like to take advantage of stratified sampling,\nso as to obtain the same target distribution in both the training and the test sets.\n"

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=43, 
                                                    stratify=y)

In [17]:
print("Training Set shape: {}".format(X_train.shape))
print("Test Set shape: {}".format(X_test.shape))

Training Set shape: (491, 22)
Test Set shape: (123, 22)


## Feature Scaling: Why/When

-  <span style="color: red">**REMEMBER:**</span> Not every learning models are sensitive to different feature scales! 

-  For example, in the case of Logistic Regression the vector of model parameters we come up with when we minimize the negative log-likelihood - using gradient descent (iterative) solution - is **not** affected by different feature scales, except for a constant.

-  You can convince yourself of this by computing the gradient of the negative log-likelihood using non-scaled and scaled features.

-  Other models, instead, are not invariant with respect to scalar transformations of the input (features), and leads to completely different results if features are not properly scaled (e.g., Support Vector Machines or SVM). 

## Feature Scaling: How

-  Feature scaling **cannot** be done looking at the whole dataset!

-  In other words, either you standardize (using **z-scores**) or normalize (using **min-max**) your features you **must** do it considering only the training set portion of your dataset.

-  The same scaling, then, should be applied to the test set.

In [18]:
"""
Let's use two different feature scaling strategies: standard z-scores and min-max
"""
# The following is the scikit-learn package which provides
# various preprocessing capabilities
from sklearn import preprocessing

In [19]:
# Standardizing features using z-score
std_scaler = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scaler.transform(X_train)
# Alternatively, using pure pandas:
# X_train_mean = X_train.mean()
# X_train_std = X_train.std()
# X_train_std = (X_train - X_train_mean)/X_train_std

# Normalizing features using min-max
minmax_scaler = preprocessing.MinMaxScaler().fit(X_train)
X_train_minmax = minmax_scaler.transform(X_train)
# Alternatively, using pure pandas:
# X_train_max = X_train.max()
# X_train_min = X_train.min()
# X_train_minmax = (X_train - X_train_min)/(X_train_max - X_train_min)

At this stage, we can work with 3 different feature matrices:
- The original one: X_train
- The standardized one: X_train_std
- The min-max normalized one: X_train_minmax

In the following, however, we work only on the original feature matrix X_train


In [20]:
"""
General function used to assess the quality of predictions
in terms of two scores: accuracy and ROC AUC (Area Under the ROC Curve)
"""
def evaluate(true_values, predicted_values):
    
    # Classification Accuracy
    print("Accuracy = {:.3f}".
          format(accuracy_score(true_values, predicted_values)))
    
    # Explained variance score: 1 is perfect prediction
    print("Area Under the ROC Curve (ROC AUC) = {:.3f}".
          format(roc_auc_score(true_values, predicted_values)))

In [21]:
# Create logistic regression object
model = LogisticRegression(solver = "liblinear")

# 1. Try to fit this logistic regressor to our original training set
model.fit(X_train, y_train)

# 2. Assess the quality of predictions made on the same training set
print("***** Evaluate Performance on Training Set *****")
evaluate(y_train, model.predict(X_train))
print()

# 3. Assess the quality of predictions made on the test set
print("***** Evaluate Performance on Test Set *****") 
evaluate(y_test, model.predict(X_test))

***** Evaluate Performance on Training Set *****
Accuracy = 0.815
Area Under the ROC Curve (ROC AUC) = 0.715

***** Evaluate Performance on Test Set *****
Accuracy = 0.789
Area Under the ROC Curve (ROC AUC) = 0.665


In [22]:
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

          -1       0.93      0.34      0.50        38
           1       0.77      0.99      0.87        85

    accuracy                           0.79       123
   macro avg       0.85      0.67      0.68       123
weighted avg       0.82      0.79      0.75       123



In [23]:
# Simplest usage of cross-validation
model = LogisticRegression(solver = "liblinear")
cv = cross_validate(model, X, y, cv=10, scoring=('roc_auc', 'accuracy'), return_train_score=True)
pd.DataFrame(cv)

,fit_time,score_time,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy
0,0.005184,0.003141,0.780906,0.794470,0.806452,0.809783
1,0.004409,0.002310,0.813953,0.793509,0.822581,0.807971
2,0.003620,0.002345,0.733333,0.804269,0.774194,0.813406
3,0.003116,0.002129,0.682143,0.809517,0.758065,0.817029
4,0.003290,0.001648,0.684211,0.809796,0.770492,0.813743
5,0.003148,0.001692,0.793233,0.795330,0.786885,0.811935
6,0.002454,0.001956,0.865915,0.787511,0.868852,0.802893
7,0.002739,0.001408,0.796992,0.798312,0.836066,0.806510
8,0.002943,0.001449,0.726817,0.803164,0.803279,0.806510
9,0.002285,0.001613,0.697995,0.806434,0.836066,0.810127


In [24]:
# Model evaluation using cross-validation
print("***** Evaluate Average Performance on Training Set *****")
print("Avg. Training Set Accuracy = {:.3f}".format(np.mean(cv['train_accuracy'])))
print("Avg. Training Set ROC AUC = {:.3f}".format(np.mean(cv['train_roc_auc'])))
print()
print("***** Evaluate Average Performance on Cross-Validation Set *****")
print("Avg. Test Set Accuracy = {:.3f}".format(np.mean(cv['test_accuracy'])))
print("Avg. Test Set ROC AUC = {:.3f}".format(np.mean(cv['test_roc_auc'])))

***** Evaluate Average Performance on Training Set *****
Avg. Training Set Accuracy = 0.810
Avg. Training Set ROC AUC = 0.800

***** Evaluate Average Performance on Cross-Validation Set *****
Avg. Test Set Accuracy = 0.806
Avg. Test Set ROC AUC = 0.758


In [25]:
# Define an object of type KFold and pass it to the cross_validate function
model = LogisticRegression(solver = "liblinear")

k_fold = KFold(n_splits=10, shuffle=True, random_state=42)

cv = cross_validate(model, X, y, cv=k_fold, scoring=('roc_auc', 'accuracy'), return_train_score=True)
print(cv)

{'fit_time': array([0.00644803, 0.00441599, 0.00330186, 0.00346279, 0.00332403,
       0.00282216, 0.00271916, 0.00264621, 0.00253701, 0.00281   ]), 'score_time': array([0.00335908, 0.00247717, 0.00207114, 0.00259924, 0.00187087,
       0.00167704, 0.00157166, 0.00159597, 0.00154328, 0.00141191]), 'test_roc_auc': array([0.80255517, 0.6845039 , 0.825     , 0.79559364, 0.70144928,
       0.78571429, 0.75062657, 0.7248062 , 0.84404762, 0.75942029]), 'train_roc_auc': array([0.79542908, 0.81216494, 0.79226979, 0.79738283, 0.80811997,
       0.7958777 , 0.80009127, 0.80634155, 0.79204556, 0.79742157]), 'test_accuracy': array([0.75806452, 0.80645161, 0.79032258, 0.83870968, 0.81967213,
       0.85245902, 0.81967213, 0.75409836, 0.80327869, 0.83606557]), 'train_accuracy': array([0.81521739, 0.80978261, 0.8115942 , 0.80615942, 0.81012658,
       0.80831826, 0.80650995, 0.81555154, 0.80831826, 0.80650995])}


In [26]:
k_fold

KFold(n_splits=10, random_state=42, shuffle=True)

In [27]:
# Model evaluation using cross-validation
print("***** Evaluate Average Performance on Training Set *****")
print("Avg. Training Set Accuracy = {:.3f}".format(np.mean(cv['train_accuracy'])))
print("Avg. Training Set ROC AUC = {:.3f}".format(np.mean(cv['train_roc_auc'])))
print()
print("***** Evaluate Average Performance on Cross-Validation Set *****")
print("Avg. Test Set Accuracy = {:.3f}".format(np.mean(cv['test_accuracy'])))
print("Avg. Test Set ROC AUC = {:.3f}".format(np.mean(cv['test_roc_auc'])))

***** Evaluate Average Performance on Training Set *****
Avg. Training Set Accuracy = 0.810
Avg. Training Set ROC AUC = 0.800

***** Evaluate Average Performance on Cross-Validation Set *****
Avg. Test Set Accuracy = 0.808
Avg. Test Set ROC AUC = 0.767


In [28]:
# Define an object of type StratifiedKFold and pass it to the cross_validate function
model = LogisticRegression(solver = "liblinear")

k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=37)

cv = cross_validate(model, X, y, cv=k_fold, scoring=('roc_auc', 'accuracy'), return_train_score=True)
print(cv)

{'fit_time': array([0.00363898, 0.00260901, 0.00259924, 0.00246811, 0.00279808,
       0.00234008, 0.00266171, 0.00237107, 0.00246501, 0.00239396]), 'score_time': array([0.00183606, 0.00165892, 0.00176287, 0.00139594, 0.00183988,
       0.00145006, 0.00148296, 0.00206113, 0.00142908, 0.00209904]), 'test_roc_auc': array([0.7246022 , 0.74051408, 0.71428571, 0.78571429, 0.77318296,
       0.74686717, 0.77443609, 0.75814536, 0.69799499, 0.81954887]), 'train_roc_auc': array([0.80519164, 0.80442906, 0.80417687, 0.79840881, 0.79577122,
       0.79796167, 0.79668391, 0.80141466, 0.80503499, 0.79440219]), 'test_accuracy': array([0.85483871, 0.80645161, 0.79032258, 0.77419355, 0.75409836,
       0.83606557, 0.7704918 , 0.81967213, 0.78688525, 0.86885246]), 'train_accuracy': array([0.80434783, 0.8115942 , 0.80978261, 0.8134058 , 0.81735986,
       0.80650995, 0.81374322, 0.80650995, 0.80831826, 0.80470163])}


In [29]:
# Model evaluation using cross-validation
print("***** Evaluate Average Performance on Training Set *****")
print("Avg. Training Set Accuracy = {:.3f}".format(np.mean(cv['train_accuracy'])))
print("Avg. Training Set ROC AUC = {:.3f}".format(np.mean(cv['train_roc_auc'])))
print()
print("***** Evaluate Average Performance on Cross-Validation Set *****")
print("Avg. Test Set Accuracy = {:.3f}".format(np.mean(cv['test_accuracy'])))
print("Avg. Test Set ROC AUC = {:.3f}".format(np.mean(cv['test_roc_auc'])))

***** Evaluate Average Performance on Training Set *****
Avg. Training Set Accuracy = 0.810
Avg. Training Set ROC AUC = 0.800

***** Evaluate Average Performance on Cross-Validation Set *****
Avg. Test Set Accuracy = 0.806
Avg. Test Set ROC AUC = 0.754


## Model Selection and Evaluation

-  So far, we have just focused on a very specific _instance_ of a Logistic Regression model.

-  In other words, we haven't spent time trying to _tune_ any "meta-parameter" (known as **hyperparameter**) of our model.

-  In practice, we used default values of hyperparameters for our Logistic Regression model, according to <code>**scikit-learn**</code>

-  We didn't perform any actual model selection, as hyperparameters are fixed!

-  The figures we output for test accuracy/ROC AUC scores are our estimates of _generalization_ performance of our model (i.e., evaluation)

## Model Selection and Evaluation (cont'd)

-  Most of the time, though, we may need to do one of the following:
    1.  Fix a "family" of models (e.g., Logistic Regression) and perform hyperparameter selection;
    2.  Choose between a set of models (e.g., Logistic Regression, SVM, Decision Tree, etc.), each one with a fixed (i.e., default) set of hyperparameters;
    3.  A mixture of the above, where we have to select the best hyperparameters of the best model picked from a set of different models.

-  In any case, we also need to provide an estimate of the generalization performance of the chosen model.

## Case 1: Select Best Hyperparameters of a Fixed Family of Models

## 1.1: Using Validation Set

In [30]:
models_and_hyperparams = {'LogisticRegression': (LogisticRegression(solver = "liblinear"),
                                                 {'C': [0.01, 0.05, 0.1, 0.5, 1, 2]}
                                                )
                         }

In [31]:
# Outer splitting: Training vs. Test set (e.g., 80÷20) used to separate training-selection-evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=73, 
                                                    stratify=y)

# Inner splitting (i.e., within the outer training set): Training vs. Validation (e.g., 80÷20)
# Training set is used to train the model, validation set is used to select the best hyperparameters
X_train_train, X_validation, y_train_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=y_train)

In [32]:
# Keep the training score obtained with each hyperparameter
training_scores = {}
# Keep the validation score obtained with each hyperparameter
validation_scores = {}
# Keep only the best training/validation scores
best_training_score = {}
best_validation_score = {}

# Get the only model available
model = models_and_hyperparams['LogisticRegression'][0]
# Get the hyperparameters for that model
hyperparams = models_and_hyperparams['LogisticRegression'][1]

# Loop through all the hyperparameters
for hp in hyperparams:
    training_scores[hp] = {}
    validation_scores[hp] = {}
    
    # Loop through all the value of a specific hyperparameter
    for val in hyperparams[hp]:
        # set the model's hyperparameter to the current value
        model.set_params(**{hp: val})
        
        # fit the model on the inner training portion 
        model.fit(X_train_train, y_train_train)
        
        # store the inner training score
        training_score = accuracy_score(y_train_train, model.predict(X_train_train))
        training_scores[hp][val] = training_score
        
        # store the inner validation score
        validation_score = accuracy_score(y_validation, model.predict(X_validation))
        validation_scores[hp][val] = validation_score
        
        # Update best training/validation scores
        if not best_training_score:
            best_training_score[hp] = (val, training_score)
        else:
            if best_training_score[hp][1] < training_score:
                best_training_score[hp] = (val, training_score)
                
        if not best_validation_score:
            best_validation_score[hp] = (val, validation_score)
        else:
            if best_validation_score[hp][1] < validation_score:
                best_validation_score[hp] = (val, validation_score)

In [33]:
print("***** Evaluate Performance on Training Set *****")
print(training_scores)
print("***** Evaluate Performance on Validation Set *****")
print(validation_scores)
print("***** Best Accuracy Score on Training Set *****")
print(best_training_score)
print("***** Best Accuracy Score on Validation Set *****")
print(best_validation_score)

***** Evaluate Performance on Training Set *****
{'C': {0.01: 0.6862244897959183, 0.05: 0.7806122448979592, 0.1: 0.8112244897959183, 0.5: 0.8188775510204082, 1: 0.8163265306122449, 2: 0.8163265306122449}}
***** Evaluate Performance on Validation Set *****
{'C': {0.01: 0.6868686868686869, 0.05: 0.7575757575757576, 0.1: 0.7676767676767676, 0.5: 0.797979797979798, 1: 0.7878787878787878, 2: 0.797979797979798}}
***** Best Accuracy Score on Training Set *****
{'C': (0.5, 0.8188775510204082)}
***** Best Accuracy Score on Validation Set *****
{'C': (0.5, 0.797979797979798)}


In [34]:
# We set the model's hyperparameters to those leading to the best score on the validation test
best_params = dict([(list(best_validation_score.keys())[0], list(best_validation_score.values())[0][0])])
model.set_params(**best_params)

# We fit this model to the whole training set portion
model.fit(X_train, y_train)
print("***** Evaluate Performance on Training Set *****")
evaluate(y_train, model.predict(X_train))
print("***** Evaluate Performance on Test Set *****")
evaluate(y_test, model.predict(X_test))

***** Evaluate Performance on Training Set *****
Accuracy = 0.813
Area Under the ROC Curve (ROC AUC) = 0.710
***** Evaluate Performance on Test Set *****
Accuracy = 0.813
Area Under the ROC Curve (ROC AUC) = 0.712


## 1.2.a: Using Cross-Validation (Single Hyperparameter)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=73, 
                                                    stratify=y)

In [36]:
models_and_hyperparams = {'LogisticRegression': (LogisticRegression(solver = "liblinear"),
                                                 {'C': [0.01, 0.05, 0.1, 0.5, 1, 2]}
                                                )
                         }

In [37]:
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Get the only model available
model = models_and_hyperparams['LogisticRegression'][0]

# Get the hyperparameters for that model
hyperparams = models_and_hyperparams['LogisticRegression'][1]

gs = GridSearchCV(estimator=model, param_grid=hyperparams, cv=k_fold, 
                  scoring='accuracy',
                  verbose=True,
                 return_train_score=True)
gs.fit(X_train, y_train)
pd.DataFrame(gs.cv_results_)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
0,0.002206,0.000412,0.000787,0.000167,0.01,{'C': 0.01},0.68,0.693878,0.693878,0.693878,...,0.687783,0.694570,0.685520,0.690045,0.687783,0.692308,0.690045,0.690045,0.689748,0.002634
1,0.001976,0.000205,0.000619,0.000075,0.05,{'C': 0.05},0.74,0.693878,0.775510,0.795918,...,0.780543,0.766968,0.771493,0.776018,0.780543,0.776018,0.762443,0.769231,0.773027,0.006183
2,0.001885,0.000092,0.000629,0.000139,0.1,{'C': 0.1},0.82,0.734694,0.816327,0.816327,...,0.805430,0.803167,0.803167,0.803167,0.812217,0.812217,0.803167,0.800905,0.805838,0.004299
3,0.002129,0.000221,0.000695,0.000195,0.5,{'C': 0.5},0.82,0.755102,0.816327,0.816327,...,0.812217,0.814480,0.812217,0.816742,0.819005,0.816742,0.809955,0.812217,0.815343,0.003800
4,0.002080,0.000123,0.000671,0.000141,1,{'C': 1},0.82,0.755102,0.816327,0.816327,...,0.814480,0.809955,0.814480,0.814480,0.816742,0.814480,0.809955,0.814480,0.814665,0.002883
5,0.002072,0.000141,0.000650,0.000143,2,{'C': 2},0.82,0.755102,0.816327,0.816327,...,0.814480,0.816742,0.814480,0.814480,0.816742,0.819005,0.807692,0.814480,0.815570,0.003198


In [38]:
print("Best hyperparameter: {}".format(gs.best_params_))
print("Best accuracy score: {:.3f}".format(gs.best_score_))
evaluate(y_test, gs.predict(X_test))

Best hyperparameter: {'C': 0.5}
Best accuracy score: 0.813
Accuracy = 0.813
Area Under the ROC Curve (ROC AUC) = 0.712


## 1.2.b: Using Cross-Validation (Multiple Hyperparameters)

In [39]:
models_and_hyperparams = {'LogisticRegression': (LogisticRegression(solver = "liblinear"),
                                                 {'C': [0.01, 0.05, 0.1, 0.5, 1, 2],
                                                 'penalty': ['l1', 'l2']}
                                                )
                         }

In [40]:
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=31)

# Get the only model available
model = models_and_hyperparams['LogisticRegression'][0]

# Get the hyperparameters for that model
hyperparams = models_and_hyperparams['LogisticRegression'][1]

gs = GridSearchCV(estimator=model, param_grid=hyperparams, cv=k_fold, 
                  scoring='accuracy',
                  verbose=True,
                 return_train_score=True)
gs.fit(X_train, y_train)
pd.DataFrame(gs.cv_results_)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
0,0.001708,0.000421,0.000686,0.000106,0.01,l1,"{'C': 0.01, 'penalty': 'l1'}",0.66,0.673469,0.693878,...,0.690045,0.683258,0.685520,0.685520,0.690045,0.690045,0.690045,0.690045,0.687938,0.002378
1,0.001796,0.000144,0.000625,0.000129,0.01,l2,"{'C': 0.01, 'penalty': 'l2'}",0.68,0.693878,0.693878,...,0.690045,0.690045,0.699095,0.687783,0.687783,0.687783,0.687783,0.690045,0.689975,0.003207
2,0.001551,0.000166,0.000613,0.000105,0.05,l1,"{'C': 0.05, 'penalty': 'l1'}",0.80,0.816327,0.755102,...,0.814480,0.814480,0.814480,0.798643,0.805430,0.807692,0.807692,0.805430,0.808554,0.004757
3,0.001926,0.000170,0.000594,0.000151,0.05,l2,"{'C': 0.05, 'penalty': 'l2'}",0.74,0.775510,0.714286,...,0.785068,0.787330,0.776018,0.766968,0.769231,0.776018,0.773756,0.771493,0.776194,0.006094
4,0.001794,0.000257,0.000639,0.000111,0.1,l1,"{'C': 0.1, 'penalty': 'l1'}",0.80,0.816327,0.755102,...,0.814480,0.814480,0.814480,0.798643,0.805430,0.807692,0.807692,0.805430,0.808554,0.004757
5,0.001917,0.000087,0.000577,0.000045,0.1,l2,"{'C': 0.1, 'penalty': 'l2'}",0.76,0.795918,0.734694,...,0.807692,0.812217,0.816742,0.798643,0.805430,0.800905,0.805430,0.800905,0.806745,0.005426
6,0.002942,0.000503,0.000668,0.000119,0.5,l1,"{'C': 0.5, 'penalty': 'l1'}",0.80,0.816327,0.734694,...,0.821267,0.816742,0.821267,0.807692,0.809955,0.809955,0.809955,0.807692,0.813080,0.004863
7,0.002170,0.000179,0.000629,0.000068,0.5,l2,"{'C': 0.5, 'penalty': 'l2'}",0.80,0.816327,0.734694,...,0.821267,0.819005,0.819005,0.807692,0.807692,0.812217,0.809955,0.812217,0.813986,0.004592
8,0.004653,0.002396,0.000680,0.000116,1,l1,"{'C': 1, 'penalty': 'l1'}",0.80,0.816327,0.734694,...,0.823529,0.816742,0.828054,0.807692,0.805430,0.812217,0.814480,0.809955,0.815117,0.006552
9,0.002070,0.000165,0.000645,0.000135,1,l2,"{'C': 1, 'penalty': 'l2'}",0.80,0.816327,0.755102,...,0.823529,0.819005,0.821267,0.807692,0.809955,0.814480,0.812217,0.812217,0.815570,0.005040


In [41]:
print("Best hyperparameter: {}".format(gs.best_params_))
print("Best accuracy score: {:.3f}".format(gs.best_score_))
evaluate(y_test, gs.predict(X_test))

Best hyperparameter: {'C': 1, 'penalty': 'l2'}
Best accuracy score: 0.811
Accuracy = 0.813
Area Under the ROC Curve (ROC AUC) = 0.712


## Case 2: Select Best Model out of a Set of Family of Models with Fixed Hyperparameters

## Using Cross Validation

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=73, 
                                                    stratify=y)

In [43]:
models = {'LogisticRegression': LogisticRegression(solver = "liblinear", max_iter=1000),
          'LinearSVC': LinearSVC(),
          'DecisionTreeClassifier': DecisionTreeClassifier(),
          'RandomForestClassifier': RandomForestClassifier(),
          'GradientBoostingClassifier': GradientBoostingClassifier()
          # Add other families of models here...
         }

In [44]:
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
cv_scores = {}
for model_name, model in models.items():
    cv_scores[model_name] = cross_val_score(model, X_train, y_train, cv=k_fold, scoring='accuracy')

/opt/homebrew/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/homebrew/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` expl

In [45]:
cv_df = pd.DataFrame(cv_scores).transpose()
cv_df

,0,1,2,3,4,5,6,7,8,9
LogisticRegression,0.82,0.755102,0.816327,0.816327,0.816327,0.816327,0.775510,0.795918,0.857143,0.836735
LinearSVC,0.34,0.306122,0.693878,0.632653,0.306122,0.693878,0.693878,0.571429,0.551020,0.673469
DecisionTreeClassifier,0.62,0.714286,0.653061,0.714286,0.755102,0.795918,0.673469,0.632653,0.693878,0.775510
RandomForestClassifier,0.76,0.755102,0.734694,0.836735,0.795918,0.816327,0.795918,0.795918,0.775510,0.795918
GradientBoostingClassifier,0.78,0.734694,0.734694,0.775510,0.775510,0.836735,0.755102,0.816327,0.755102,0.816327


In [46]:
cv_df['avg_cv'] = np.mean(cv_df, axis=1)
cv_df['std_cv'] = np.std(cv_df, axis=1)
cv_df = cv_df.sort_values(['avg_cv', 'std_cv'], ascending=[False,True])
cv_df

,0,1,2,3,4,5,6,7,8,9,avg_cv,std_cv
LogisticRegression,0.82,0.755102,0.816327,0.816327,0.816327,0.816327,0.775510,0.795918,0.857143,0.836735,0.810571,0.026278
RandomForestClassifier,0.76,0.755102,0.734694,0.836735,0.795918,0.816327,0.795918,0.795918,0.775510,0.795918,0.786204,0.027402
GradientBoostingClassifier,0.78,0.734694,0.734694,0.775510,0.775510,0.836735,0.755102,0.816327,0.755102,0.816327,0.778000,0.031914
DecisionTreeClassifier,0.62,0.714286,0.653061,0.714286,0.755102,0.795918,0.673469,0.632653,0.693878,0.775510,0.702816,0.054060
LinearSVC,0.34,0.306122,0.693878,0.632653,0.306122,0.693878,0.693878,0.571429,0.551020,0.673469,0.546245,0.150015


In [47]:
# Model Selection: Logistic Regression is the best overall method, therefore we pick that!
# Now we need to provide an estimate of its generalization performance. 
# To do so, we evaluate it against the test set portion we previously held out.
model = models[cv_df.index[0]]
# Re-fit the best selected model on the whole training set
model.fit(X_train, y_train)
# Evaluation
print("***** Evaluate Performance on Training Set *****")
evaluate(y_train, model.predict(X_train))
print("***** Evaluate Performance on Test Set *****")
evaluate(y_test, model.predict(X_test))

***** Evaluate Performance on Training Set *****
Accuracy = 0.817
Area Under the ROC Curve (ROC AUC) = 0.717
***** Evaluate Performance on Test Set *****
Accuracy = 0.813
Area Under the ROC Curve (ROC AUC) = 0.712


## Case 3: Select the Best Hyperparameters AND the Best Model from a Family of Models

In [48]:
models_and_hyperparams = {'LogisticRegression': (LogisticRegression(),
                                                 {'C': [0.01, 0.05, 0.1, 0.5, 1, 2],
                                                 'penalty': ['l1', 'l2']}
                                                ),
                          'RandomForestClassifier': (RandomForestClassifier(),
                                       {'n_estimators': [10, 50, 100]}
                                       ),
                          'DecisionTreeClassifier': (DecisionTreeClassifier(),
                                                     {'criterion': ['gini', 'entropy'], 
                                                      'max_depth': [i for i in range(1, X.shape[1]+1)]}
                                                    )
                         }

In [49]:
# `outer_cv` creates 10 folds for estimating generalization error
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# when we train on a certain fold, we use a second cross-validation
# split in order to choose hyperparameters
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=73)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=37, 
                                                    stratify=y)

# we will collect the average of the scores on the 10 outer folds in this dictionary
# with keys given by the names of the models in `models_and_hyperparams`
average_scores_across_outer_folds_for_each_model = dict()

# find the model with the best generalization error
for name, (model, params) in models_and_hyperparams.items():
    # this object is a classifier that also happens to choose
    # its hyperparameters automatically using `inner_cv`
    model_optimizing_hyperparams = GridSearchCV(estimator=model, 
                                                param_grid=params,
                                                cv=inner_cv, 
                                                scoring='accuracy',
                                               verbose=True)

    # estimate generalization error on the 10-fold splits of the data
    scores_across_outer_folds = cross_val_score(model_optimizing_hyperparams,
                                                X_train, y_train, cv=outer_cv, scoring='accuracy')

    # get the mean accuracy across each of outer_cv's 10 folds
    average_scores_across_outer_folds_for_each_model[name] = np.mean(scores_across_outer_folds)
    performance_summary = 'Model: {name}\nAccuracy in the 10 outer folds: {scores}.\nAverage Accuracy: {avg}'
    print(performance_summary.format(
        name=name, scores=scores_across_outer_folds,
        avg=np.mean(scores_across_outer_folds)))
    print()

print('Average score across the outer folds: ',
      average_scores_across_outer_folds_for_each_model)

many_stars = '\n' + '*' * 100 + '\n'
print(many_stars + 'Now we choose the best model and refit on the whole dataset' + many_stars)

best_model_name, best_model_avg_score = max(
    average_scores_across_outer_folds_for_each_model.items(),
    key=(lambda name_averagescore: name_averagescore[1]))

# get the best model and its associated parameter grid
best_model, best_model_params = models_and_hyperparams[best_model_name]

# now we refit this best model on the whole dataset so that we can start
# making predictions on other data, and now we have a reliable estimate of
# this model's generalization error and we are confident this is the best model
# among the ones we have tried
final_model = GridSearchCV(best_model, best_model_params, cv=inner_cv)
final_model.fit(X_train, y_train)

print('Best model: \n\t{}'.format(best_model), end='\n\n')
print('Estimation of its generalization performance (accuracy):\n\t{}'.format(
    best_model_avg_score), end='\n\n')
print('Best parameter choice for this model: \n\t{params}'
      '\n(according to cross-validation `{cv}` on the whole dataset).'.format(
      params=final_model.best_params_, cv=inner_cv))


y_true, y_pred, y_pred_prob = y, final_model.predict(X), final_model.predict_proba(X)
print()
print(classification_report(y_true, y_pred))
roc = roc_auc_score(y_true, y_pred_prob[:,1])
acc = accuracy_score(y_true, y_pred)
print("Accuracy = [{:.3f}]".format(acc))
print("Area Under the ROC = [{:.3f}]".format(roc))

Fitting 10 folds for each of 12 candidates, totalling 120 fits


/opt/homebrew/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Fitting 10 folds for each of 12 candidates, totalling 120 fits


/opt/homebrew/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Fitting 10 folds for each of 12 candidates, totalling 120 fits


...

Fitting 10 folds for each of 12 candidates, totalling 120 fits


/opt/homebrew/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Model: LogisticRegression
Accuracy in the 10 outer folds: [0.76       0.89795918 0.81632653 0.81632653 0.85714286 0.85714286
 0.79591837 0.79591837 0.75510204 0.7755102 ].
Average Accuracy: 0.812734693877551

Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Model: RandomForestClassifier
Accuracy in the 10 outer folds: [0.76       0.83673469 0.79591837 0.75510204 0.7755102  0.87755102
 0.79591837 0.7755102  0.71428571 0.79591837].
Avera

/opt/homebrew/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Best model: 
	LogisticRegression()

Estimation of its generalization performance (accuracy):
	0.812734693877551

Best parameter choice for this model: 
	{'C': 1, 'penalty': 'l2'}
(according to cross-validation `StratifiedKFold(n_splits=10, random_state=73, shuffle=True)` on the whole dataset).

              precision    recall  f1-score   support

          -1       0.81      0.46      0.59       192
           1       0.80      0.95      0.87       422

    accuracy                           0.80       614
   macro avg       0.80      0.71      0.73       614
weighted avg       0.80      0.80      0.78       614

Accuracy = [0.798]
Area Under the ROC = [0.791]
